In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import os
import re

In [ ]:
def load_driver(SELENIUM_EXECUTABLE_PATH=r'/mnt/c/Users/adity/Downloads/Chrome/geckodriver-v0.27.0-win64/geckodriver.exe'):
    driver = webdriver.Firefox(executable_path=SELENIUM_EXECUTABLE_PATH)
    return driver

def save_to_csv(data, SAVE_PATH, MODE):
    flag = os.path.isfile(SAVE_PATH)
    with open(SAVE_PATH, MODE) as file:
        fileWriter = csv.DictWriter(file, fieldnames=data[0].keys(), delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        if not flag:
            fileWriter.writeheader()
        fileWriter.writerows(data)
        print ("Data Written = ", data)

def parse_population_details(population, global_identifiers, tag=None, SAVE_PATH='logs/population_report.csv', MODE='a'):
    BASE_PATH='https://www.psacard.com'
    rel_url = population['href'].split('/')[3:]
    auction_year = rel_url[0]
    population_id = rel_url[-1]
    url = BASE_PATH + population['href']
    driver = load_driver()
    driver.get(url)
    soup=BeautifulSoup(driver.page_source, features="lxml")
    all_details = soup.find_all("div", attrs={"class": "dataTables_wrapper"})[0]

    population_arr = []
    for row in all_details.findAll('tr')[1:]:
        try:
            population_row = {}
            all_td = row.findAll('td')[1:]
            metrices = [th.text.strip() for th in all_details.findAll('th')[3:]]
            auction_meta = str(all_td[0])
            population_row['auction_player'] = auction_meta[auction_meta.index('<strong>')+len('<strong>'):auction_meta.index('</strong>')].strip()
            population_row['auction_league'] = auction_meta[auction_meta.index('<br/>')+8: auction_meta.index('<span')-3].strip()
            population_row['population_id'] = population_id
            population_row['auction_year'] = auction_year
            population_row['fk_name'], population_row['fk_url'], population_row['fk_count'], population_row['fk_category'] = global_identifiers

            # Collecting all metrices for the player
            idx = 0
            for item in all_td[2:]:
                cells = item.findAll('span')
                population_row[metrices[idx]] = {'Grade': cells[0].contents[0], '+': cells[1].contents[0], 'Q': cells[2].contents[0]}
                idx += 1
            population_arr.append(population_row)
        except:
            print ("Error in population parsing ....")
            continue
    driver.quit()

    # Append to the csv file
    save_to_csv(population_arr, SAVE_PATH, MODE)
    return

def parse_amount_details(rel_url, global_identifiers, tag='None', SAVE_PATH='logs/transaction.csv', MODE='a'):
    BASE_PATH='https://www.psacard.com'
    identifiers = rel_url['href'].split('/')[3:]
    auction_name = identifiers[0]
    auction_player = identifiers[1]
    value = rel_url.text
    summary_id = re.sub(r'#.*', '', identifiers[-1])

    url = BASE_PATH + most_recent_prices['href']
    driver = load_driver()
    driver.get(url)
    soup=BeautifulSoup(driver.page_source, features="lxml")
    all_details = soup.find_all("div", attrs={"class": "item-lots-summary"})[0].find_all("div", attrs={"class": "item-row"})
    amount_arr = []
    for fine_details in all_details[1:]:
        specifics = {}
        specifics['auction_name'] = auction_name
        specifics['auction_player'] = auction_player
        specifics['value'] = value
        specifics['summary_id'] = summary_id
        specifics['date'] = fine_details['data-date']
        specifics['price'] = fine_details['data-price']
        specifics['grade'] = fine_details['data-gradevalue']
        specifics['lot_number'] = fine_details['data-lot']
        specifics['auction_house'] = fine_details['data-auctionhouse']
        specifics['auction_seller'] = fine_details['data-auctionname']
        specifics['auction_type'] = fine_details['data-auctiontype']
        specifics['cert'] = fine_details['data-cert']
        specifics['tag'] = tag

        specifics['fk_name'], specifics['fk_url'], specifics['fk_count'], specifics['fk_category'] = global_identifiers
        amount_arr.append(specifics)

    # Save to File
    save_to_csv(amount_arr, SAVE_PATH, MODE)
    driver.quit()
    return

In [ ]:
cnt = 0
CSV_FILE_PATH='logs/allauctionprices.csv'
with open(CSV_FILE_PATH) as csvfile:
        auction_items= csv.reader(csvfile, delimiter=',')
        for row in auction_items:
            cnt += 1
            if cnt > 5:
                break
            name, url, count, category = row
            if len(url) < 10:
                continue
            global_identifiers = [name, url, count, category]
            driver = load_driver()
            driver.get(url)
            soup=BeautifulSoup(driver.page_source, features="lxml")
            grade_prices = soup.find_all("table", attrs={"class": "set-items-results"})
            driver.quit()
            for idx in range(len(grade_prices)-1):
                specifics = grade_prices[idx].findAll('tr')
                grade = specifics[0].findAll('td')[-1].contents[0]
                most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
                average_price = specifics[2].findAll('td')[-1].contents[0]
                smr_price = specifics[3].findAll('td')[-1].contents[0]
                population = specifics[4].findAll('td')[-1].contents[0]
                pop_higher = specifics[5].findAll('td')[-1].contents[0]

                if str(most_recent_prices) != '—':
                    parse_amount_details(most_recent_prices, global_identifiers, 'Most Recent Price')

                if str(average_price) != '—':
                    parse_amount_details(average_price, global_identifiers, 'Average Price')

                if str(smr_price) != '—':
                    parse_amount_details(most_recent_prices, global_identifiers, 'SMR Price' )

In [ ]:
url

In [ ]:
def main():
    CSV_FILE_PATH='logs/allauctionprices.csv'
    cnt = 0
    with open(CSV_FILE_PATH) as csvfile:
        auction_items= csv.reader(csvfile, delimiter=',')
        for row in auction_items:
            try:
                name, url, count, category = row
                global_identifiers = [name, url, count, category]
                driver = load_driver()
                driver.get(url)
                soup=BeautifulSoup(driver.page_source, features="lxml")
                grade_prices = soup.find_all("table", attrs={"class": "set-items-results"})
                for idx in range(len(grade_prices)-1):
                    specifics = grade_prices[idx].findAll('tr')
                    grade = specifics[0].findAll('td')[-1].contents[0]
                    most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
                    average_price = specifics[2].findAll('td')[-1].contents[0]
                    smr_price = specifics[3].findAll('td')[-1].contents[0]
                    population = specifics[4].findAll('td')[-1].contents[0]
                    pop_higher = specifics[5].findAll('td')[-1].contents[0]

                    if str(most_recent_prices) != '—':
                        parse_amount_details(most_recent_prices, global_identifiers, 'Most Recent Price')

                    if str(average_price) != '—':
                        parse_amount_details(average_price, global_identifiers, 'Average Price')

                    if str(smr_price) != '—':
                        parse_amount_details(most_recent_prices, global_identifiers, 'SMR Price' )

                    # if str(population) != '—':
                    #     parse_population_details(population, global_identifiers, 'Population')
                    cnt += 1
                    if cnt > 5:
                        break

                driver.quit()
            except:
                continue

if __name__ == '__main__':
    main()


In [ ]:
def main():
    CSV_FILE_PATH='logs/allauctionprices.csv'
    cnt = 0
    with open(CSV_FILE_PATH) as csvfile:
        auction_items= csv.reader(csvfile, delimiter=',')
        for row in auction_items:
            try:
                name, url, count, category = row
                global_identifiers = [name, url, count, category]
                driver = load_driver()
                driver.get(url)
                soup=BeautifulSoup(driver.page_source, features="lxml")
                grade_prices = soup.find_all("table", attrs={"class": "set-items-results"})
                for idx in range(len(grade_prices)-1):
                    specifics = grade_prices[idx].findAll('tr')
                    grade = specifics[0].findAll('td')[-1].contents[0]
                    most_recent_prices = specifics[1].findAll('td')[-1].contents[0]
                    average_price = specifics[2].findAll('td')[-1].contents[0]
                    smr_price = specifics[3].findAll('td')[-1].contents[0]
                    population = specifics[4].findAll('td')[-1].contents[0]
                    pop_higher = specifics[5].findAll('td')[-1].contents[0]

                    if str(most_recent_prices) != '—':
                        parse_amount_details(most_recent_prices, global_identifiers, 'Most Recent Price')

                    if str(average_price) != '—':
                        parse_amount_details(average_price, global_identifiers, 'Average Price')

                    if str(smr_price) != '—':
                        parse_amount_details(most_recent_prices, global_identifiers, 'SMR Price' )

                    # if str(population) != '—':
                    #     parse_population_details(population, global_identifiers, 'Population')
                    cnt += 1
                    if cnt > 5:
                        break

                driver.quit()
            except:
                continue

if __name__ == '__main__':
    main()
